In [ ]:
import html
from pyspark.sql import functions as f
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame

spark = SparkSession.builder.getOrCreate()

RAW_PATH = "/home/jovyan/data-sets/sentiment-140-training-data/RAW"
CLEAN_PATH = "/home/jovyan/data-sets/sentiment-140-training-data/CLEAN"

### Findings from our Data Exploration step:

Re-ordered in the order in which we will pick these up:

- __Problem:__ We need to apply a proper schema
    > __Action:__ create `spark_reader` by applying `.schema()` to `SparkSession` object with proper schema DDL defined

- __Problem:__ Encoding seems to be 'broken' - example: 
```markdown
�����ߧ�ǿ�����ж�؜��� &lt;&lt;----I DID NOT KNOW I CUD or HOW TO DO ALL DAT ON MY PHONE TIL NOW. WOW..MY LIFE IS NOW COMPLETE. JK.
```
    > __Action:__ remove broken chars. Or, more rigorous, replace any none letter with a blank space << what happens when we do this?

- __Problem:__ The date column needs fixing
    > __Action:__ set `timestampFormat` option while reading csv

- We need to extract twitter user names/handles (we'll extract it and call the output column users_mentioned)
- We need to extract hashtags and replace them with the words from the hashtag (we'll extract it and call the output column hashtags)
- We need to extract URLs, as our algorithm won't need that or use that (we'll simply remove it from the data)
- The same goes for email-address
- HTML does not appear properly unescaped, we're going to have to fix that (example: &lt;3 and s&amp;^t)


...



In [ ]:
# Have a first look at the data
raw_data = spark.read.csv(PATH)  # .cache()

# Let's look at 50 rows of data
raw_data.show(50, False)

## Apply Proper Schema

Let's start with applying the proper schema.

From the documentation (see README):
> The data is a CSV with emoticons removed. Data file format has 6 fields:  
0 - the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)  
1 - the id of the tweet (2087)  
2 - the date of the tweet (Sat May 16 23:58:44 UTC 2009)  
3 - the query (lyx). If there is no query, then this value is NO_QUERY.  
4 - the user that tweeted (robotickilldozr)  
5 - the text of the tweet (Lyx is cool)  

Let's apply the learnings from our Data Exploration phase and make a proper schema DDL.  
We will create a new SparkReader and store it in spark_reader to be used by our next steps.

In [ ]:
schema_ddl = """
polarity STRING COMMENT "the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)",
id LONG COMMENT "the id of the tweet (2087)",
date TIMESTAMP COMMENT "the date of the tweet (Sat May 16 23:58:44 UTC 2009)",
query STRING COMMENT "the query (lyx). If there is no query, then this value is NO_QUERY.",
user string COMMENT "the user that tweeted (robotickilldozr)",
text string COMMENT "the text of the tweet (Lyx is cool)"
"""

spark_reader = spark.read.schema(schema_ddl)

## Converting Date column

Next, we will convert the date column to `TimestampType` by applying `timestampFormat` to our `spark_reader.csv()` method 

We will use Java SimpleDateFormat: https://docs.oracle.com/javase/8/docs/api/java/text/SimpleDateFormat.html

Based on the sample provided (`Sat May 16 23:58:44 UTC 2009`) we can determine that the correct format should be:
```java
"EEE MMM dd HH:mm:ss zzz yyyy"
```

In [ ]:
simple_date_format = "EEE MMM dd HH:mm:ss zzz yyyy"

raw_data = spark_reader.csv(RAW_PATH, timestampFormat=simple_date_format)
raw_data.show()

raw_data.summary().show()

## Cleaning the tweet text

We're going to have to do the following to the data in the text column:
- Remove email-addresses and URLs
- Extract and then remove user-names (@mentions)
- Extract and then remove hash-tags (#hash-tag)

### Generate some test data

In [ ]:
# Let's start with getting some data to test our RegEx on
raw_data.select("text").show(50, False)

It looks like the top 50 rows of data have @-mentions, #-hashtags, and some URLs. No email-addresses are present.  
Let's assume that email-addresses also exist in our data, and design accordingly.

Assuming this, we can generate a few lines of test data to apply our RegEx on containing some valid and invalid cases, and we can add some email-addresses and different style URLs as wel:
```
?Obama Administration Must Stop Bonuses to AIG Ponzi Schemers ... http://bit.ly/2CUIg
@spinuzzi: Has been a bit crazy, with steep learning curve, but LyX is really good for long docs. For anything shorter, it would be insane.
@phyreman9 Google is always a good place to look. Should've mentioned I worked on the Mustang w/ my Dad, @KimbleT.
[#MLUC09] Customer Innovation Award Winner: Booz Allen Hamilton -- http://ping.fm/c2hPP
#lebron best athlete of our generation, if not all time (basketball related) I don't want to get into inter-sport debates about   __1/2
localhost:1234
some@email.adress
some.email@domain.com
a_much_more_complicated_email.some.email@domain.with.lots.of.dots.and_underscores.longdomainname
www.something.com
http://spark.apache.org/
file://some_file_location/file.fmt
file:///some_file_location/file.fmt
https://blip.fm/~6emhv
https://www.freeformatter.com/java-regex-tester.html#ad-output
https://spark.apache.org/docs/2.4.3/api/python/pyspark.ml.html#module-pyspark.ml.evaluation
google@gmail.com
```


> __Note__ Spark uses Java style RegEx which is not neccesarily the same as Python.

I like to use this website for designing and testing RegEx for Spark.:  
https://www.freeformatter.com/java-regex-tester.html


### Extracting/Removing UserNames:

Let's start with Usernames

According to twitter
> Your username cannot be longer than 15 characters. Your real name can be longer (20 characters), but usernames are kept shorter for the sake of ease. A username can only contain alphanumeric characters (letters A-Z, numbers 0-9) with the exception of underscores, as noted above. Check to make sure your desired username doesn't contain any symbols, dashes, or spaces.

So now let's design a Regex for doing just this.


In [ ]:
# extract twitter user names/handles to the output column `user_mentioned`

user_regex = r"(@\w{1,15})"

raw_data.select(
    f.regexp_extract(f.col("text"), twitter_mention_regex, 1).alias("user_mentioned"),
    "text",
).show(50, False)

A limitation that we have here though is that it only extracts the first mention of a user. This presents a limitation of how Spark works with `regexp_extract`.

You can see in this Jira Improvement a request to add a `regexp_match_all` method: https://issues.apache.org/jira/browse/SPARK-24884


In the interim, we have to run our `regexp_match` multiple times. An example:


In [ ]:
# extract up to 6 twitter user names/handles to the output column `users_mentioned`

user_regex = r"(@\w{1,15})"

raw_data.select(
    f.array_remove(
        f.array(
            f.regexp_extract(f.col("text"), user_regex, 1),
            f.regexp_extract(
                f.col("text"), "".join([f"{user_regex}.*?" for i in range(0, 2)]), 2
            ),
            f.regexp_extract(
                f.col("text"), "".join([f"{user_regex}.*?" for i in range(0, 3)]), 3
            ),
            f.regexp_extract(
                f.col("text"), "".join([f"{user_regex}.*?" for i in range(0, 4)]), 4
            ),
            f.regexp_extract(
                f.col("text"), "".join([f"{user_regex}.*?" for i in range(0, 5)]), 5
            ),
            f.regexp_extract(
                f.col("text"), "".join([f"{user_regex}.*?" for i in range(0, 6)]), 6
            ),
        ),
        "",
    ).alias("users_mentioned"),
    "text",
).toPandas().head(35)


Of course, one could code this into a function, something like this

In [ ]:
def regexp_extract_all(
    df: DataFrame,
    regex: str,
    no_of_extracts: int,
    input_column_name: str,
    output_column_name: str = "output",
    empty_array_replace: bool = True,
):
    """Pyspark implementation for extracting all matches of a reg_exp_match
    
    Parameters
    ----------
    df: DataFrame
        Input DataFrame
    
    regex: str
        Regexp string to extract from input DataFrame
    
    no_of_extracts: int
        Max number of occurrences to extract
    
    input_column_name: str
        Name of the input column
    
    output_column_name: str
        Name of the output column (default: output)
    
    empty_array_replace: bool
        If set to True, will replace empty arrays with null values (default: True)
    """
    repeats = range(0, no_of_extracts)
    
    # A set of interim columns are created that will be dropped afterwards
    match_columns = [f"___{r}___" for r in repeats]
    
    # Apply regexp_extract an r number of times
    for r in repeats:
        df = df.withColumn(
            match_columns[r],
            f.regexp_extract(
                f.col(input_column_name),
                # the input regex string is amended with ".*?" 
                # and repeated an r number of times
                # r needs to be +1 as matching groups are 1-index
                "".join([f"{regex}.*?" for i in range(0, r+1)]),
                r+1,
            ),
        )

    # Create a distinct array, all empty strings removed
    df = df.withColumn(
        output_column_name,
        f.array_remove(f.array_distinct(f.array(match_columns)), ""),
    )

    # Replace empty string with None if empty_array_replace was set 
    if empty_array_replace:
        df = df.withColumn(
            output_column_name,
            f.when(f.size(output_column_name) == 0, f.lit(None)).otherwise(
                f.col(output_column_name)
            ),
        )
    
    # Drop interim columns
    for c in match_columns:
        df = df.drop(c)

    return df


# Example usage
regexp_extract_all(raw_data, user_regex, 6, "text", "users_mentioned", True).select(
    "text", "users_mentioned"
).show(35, False)

But... we don't just want to extract the mentioned user, we want to also remove it from the text. Let's do that next using `regexp_replace`

In [ ]:
raw_data.select(
    f.regexp_replace(f.col("text"), user_regex, "").alias("text"),
    f.col("text").alias("original_text"),
).toPandas().head(20)

This little test shows that our RegEx is working. However, we do have one side-effect of this relatively simplistic approach, which is that the `...@domain.` part of email-addresses are also seen as hashtags. For now, I will accept that bug, because we can remove Emails and URLs first. Meaning, we can work around this by applying right order of operation, rather than complicating our RegEx even further.

### Extracting hashtags

Now let's repeat the process, but this time to extract and remove hashtags.

Hashtags follow a very similar approach to @-mentions, only now we use the `#`-character in place of the `@`-symbol.

This is what Twitter has to see about hashtags: https://help.twitter.com/en/using-twitter/how-to-use-hashtags  
- There is no mention of a maximum length a hashtag can be. 

So, we can modify our `user_regex` accordingly:

In [ ]:
hashtag_regex = "(#\w{1,})"

temp_df = raw_data.filter(f.instr(f.col("text"), "#") > 4)

# Let's use the function we just created to extract the hashtags
_ = regexp_extract_all(temp_df, hashtag_regex, 8, "text", "hashtags", True).select(
    "text", "hashtags"
)

_.show(35, False)

Now that we extracted the hashtags, let's convert them to regular words. We start by slightly modifying the Regex, so that the matching group is now outside the hashtag, then we simply apply a `regexp_replace`, where we replace the found hashtag with a version of itself without a hashtag added.

In [ ]:
hashtag_replace_regex = "#(\w{1,})"

_.select(f.regexp_replace(f.col("text"), hashtag_replace_regex, "$1"), "hashtags").show(35, False)

### Removing URLs and Emails

Now let's repeat the process, but this time to remove URLs and Emails.

In [ ]:
url_regex=r"((https?|ftp|file):\/{2,3})+([-\w+&@#/%=~|$?!:,.]*)|(www.)+([-\w+&@#/%=~|$?!:,.]*)"
email_regex=r"[\w.-]+@[\w.-]+\.[a-zA-Z]{1,}"

In [ ]:
raw_data.select(
    f.regexp_replace(f.col("text"), email_regex, "").alias("text_no_email"),
    f.regexp_replace(f.col("text"), url_regex, "").alias("text_no_url"),
    f.col("text").alias("original_text"),
).toPandas().head(20)

## HTML Decoding

Next up, let's look at the HTML unescaping issue.

When looking at the raw text, the HTML does not appear properly unescaped, we're going to have to fix that.  
examples of the issue:
```
started to think that Citi is in really deep s&amp;^t. Are they gonna survive the turmoil or are they gonna be the next AIG?
"I'm listening to ""P.Y.T"" by Danny Gokey &lt;3 &lt;3 &lt;3 Aww
```

We can do this by creating a UDF (User Defined Function)

In [ ]:
from pyspark.sql.functions import udf
import html

@udf
def html_unescape(s: str):
    return html.unescape(s)

raw_data.select(html_unescape("text")).show(35, False)

As you can see, no more weird HTML escape characters any more :)

## Assemble your cleaning process

So now that we have applied our individual fixes, let's assemble our cleaning process

In [ ]:
def regexp_extract_all(
    df: DataFrame,
    regex: str,
    no_of_extracts: int,
    input_column_name: str,
    output_column_name: str = "output",
    empty_array_replace: bool = True,
):
    """Pyspark implementation for extracting all matches of a reg_exp_match
    
    Parameters
    ----------
    df: DataFrame
        Input DataFrame
    
    regex: str
        Regexp string to extract from input DataFrame
    
    no_of_extracts: int
        Max number of occurrences to extract
    
    input_column_name: str
        Name of the input column
    
    output_column_name: str
        Name of the output column (default: output)
    
    empty_array_replace: bool
        If set to True, will replace empty arrays with null values (default: True)
    """
    repeats = range(0, no_of_extracts)

    # A set of interim columns are created that will be dropped afterwards
    match_columns = [f"___{r}___" for r in repeats]

    # Apply regexp_extract an r number of times
    for r in repeats:
        df = df.withColumn(
            match_columns[r],
            f.regexp_extract(
                f.col(input_column_name),
                # the input regex string is amended with ".*?"
                # and repeated an r number of times
                # r needs to be +1 as matching groups are 1-indexed
                "".join([f"{regex}.*?" for i in range(0, r + 1)]),
                r + 1,
            ),
        )

    # Create a distinct array, all empty strings removed
    df = df.withColumn(
        output_column_name,
        f.array_remove(f.array_distinct(f.array(match_columns)), ""),
    )

    # Replace empty string with None if empty_array_replace was set
    if empty_array_replace:
        df = df.withColumn(
            output_column_name,
            f.when(f.size(output_column_name) == 0, f.lit(None)).otherwise(
                f.col(output_column_name)
            ),
        )

    # Drop interim columns
    for c in match_columns:
        df = df.drop(c)

    return df


@udf
def html_unescape(s: str):
    if isinstance(s, str):
        return html.unescape(s)
    return s


def cleaning_process(df: DataFrame):
    url_regex = r"((https?|ftp|file):\/{2,3})+([-\w+&@#/%=~|$?!:,.]*)|(www.)+([-\w+&@#/%=~|$?!:,.]*)"
    email_regex = r"[\w.-]+@[\w.-]+\.[a-zA-Z]{1,}"
    user_regex = r"(@\w{1,15})"
    hashtag_regex = "(#\w{1,})"
    hashtag_replace_regex = "#(\w{1,})"

    # Cleaning process:

    # 1. Extract user mentions and hashtags
    df = regexp_extract_all(df, user_regex, 6, "text", "users_mentioned", True)
    df = regexp_extract_all(df, hashtag_regex, 6, "text", "hashtags", True)

    # 2. Remove email addresses, URLs, and user mentions
    # Order is important here
    df = (
        df
        .withColumn("original_text", f.col("text"))
        .withColumn("text", f.regexp_replace(f.col("text"), url_regex, ""))
        .withColumn("text", f.regexp_replace(f.col("text"), email_regex, ""))
        .withColumn("text", f.regexp_replace(f.col("text"), user_regex, ""))
    )

    # 3. Extract words from hashtags
    df = df.withColumn(
        "text", f.regexp_replace(f.col("text"), hashtag_replace_regex, "$1")
    )

    # 4. Unescape HTML
    df = df.withColumn("text", html_unescape("text"))

    # 5. Drop all rows with no data in text after cleaning steps were applied
    # df = df.filter(f.col("text") != "").na.drop(subset="text")
    return df

schema = """
polarity STRING COMMENT "the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)",
id INT COMMENT "the id of the tweet (2087)",
date STRING COMMENT "the date of the tweet (Sat May 16 23:58:44 UTC 2009)",
query STRING COMMENT "the query (lyx). If there is no query, then this value is NO_QUERY.",
user string COMMENT "the user that tweeted (robotickilldozr)",
text string COMMENT "the text of the tweet (Lyx is cool)"
"""

raw_data = spark.read.schema(schema).csv(RAW_PATH)
clean_data = cleaning_process(raw_data)
clean_data.show()
clean_data.select("text").show(250, False)



In [ ]:
raw_data.count()

In [ ]:
clean_data.count()

In [ ]:
clean_data.filter("text == ''").show(1000)

In [ ]:
clean_data.write.parquet(CLEAN_PATH)